In [1]:
import pynvml as N

In [2]:
def _decode(b):
            if isinstance(b, bytes):
                return b.decode('utf-8')    # for python3, to unicode
            return b

In [3]:
N.nvmlInit()
handle = N.nvmlDeviceGetHandleByIndex(0)

In [4]:
handle

In [5]:
name = _decode(N.nvmlDeviceGetName(handle))
uuid = _decode(N.nvmlDeviceGetUUID(handle))

In [6]:
name

'Tesla P100-PCIE-12GB'

In [7]:
uuid

'GPU-b2dd4fad-98f0-c52a-bf2f-8bacdfade52f'

In [8]:
memory = N.nvmlDeviceGetMemoryInfo(handle)

In [9]:
memory

In [10]:
print(memory)

AttributeError: module 'string' has no attribute 'join'

In [14]:
memory.used/(1024*1024)

12099.5

In [18]:
from core import GPUStatCollection

In [19]:
gpu_stats = GPUStatCollection.new_query()

In [20]:
gpu_stats

GPUStatCollection(host=p100, [
  [0] Tesla P100-PCIE-12GB | 58°C,  99 % | 12099 / 12198 MB | gsx(4111M) gsx(3989M) gsx(3989M)
  [1] Tesla P100-PCIE-12GB | 61°C,  99 % | 11512 / 12198 MB | gsx(2467M) gsx(2467M) gsx(2467M) gsx(4101M)
  [2] Tesla P100-PCIE-12GB | 79°C,  94 % | 11793 / 12198 MB | mzy(11779M)
  [3] Tesla P100-PCIE-12GB | 67°C, 100 % |  9858 / 12198 MB | pfren(2463M) gsx(2457M) pfren(2465M) pfren(2463M)
])

In [26]:
gpu_stats.gpus[0].memory_used

12099

In [27]:
gpu_stats

GPUStatCollection(host=p100, [
  [0] Tesla P100-PCIE-12GB | 58°C,  99 % | 12099 / 12198 MB | gsx(4111M) gsx(3989M) gsx(3989M)
  [1] Tesla P100-PCIE-12GB | 61°C,  99 % | 11512 / 12198 MB | gsx(2467M) gsx(2467M) gsx(2467M) gsx(4101M)
  [2] Tesla P100-PCIE-12GB | 79°C,  94 % | 11793 / 12198 MB | mzy(11779M)
  [3] Tesla P100-PCIE-12GB | 67°C, 100 % |  9858 / 12198 MB | pfren(2463M) gsx(2457M) pfren(2465M) pfren(2463M)
])

In [38]:
a = ['0','1','2']

In [39]:
a.remove('1')

In [40]:
a

['0', '2']

In [43]:
sckey = 'SCU117143T6c8ab81691bec379fc7edfbbac32526c5f7eb71836b34'
empty_card = 0
s = f'gpu stats: \n\n'
for i in gpu_stats.gpus:
    s+= f'gpu {i.index}: | memory used {i.memory_used}/{i.memory_total} | users: '
    for p in i.processes:
        username = p['username']
        mem = p['gpu_memory_usage']
        s+= f'{username} use {mem}|'
    s+= '\n\n'
    
sc_res_raw = requests.post(
            f'https://sc.ftqq.com/{sckey}.send',
            data={
                'text': f'空闲的GPU是{empty_card}',
                'desp': f'{s}',
            },
            timeout=1,)
print(tsc_res_raw)

<Response [200]>


In [45]:
print(sc_res_raw.text)

{"errno":0,"errmsg":"success","dataset":"done"}


In [ ]:
import requests
from core import GPUStatCollection
import json

sckey = 'SCU117143T6c8ab81691bec379fc7edfbbac32526c5f7eb71836b34'
empty_card = []
flag = True # 避免重复发送gpu推送


gpu_stats = GPUStatCollection.new_query()
for i in gpu_stats.gpus:
    if i.memory_used < 100:
        if i.index not in empty_card:
            empty_card.append(i.index)
            flag = True
    else:
        if i.index in empty_card:
            empty_card.remove(i.index)
            

if (empty_card is not None) and (flag):
    push_to_wechat()
    flag = False


def push_to_wechat():
    
    sc_res_raw = requests.post(
            f'https://sc.ftqq.com/{sckey}.send',
            data={
                'text': f'空闲的GPU是{empty_card}',
                'desp': f'{gpu_stats}',
            },
            timeout=1,
        )
    try:
        return_json = json.loads(sc_res_raw)
    except:
        raise RuntimeError(f'Server 酱的返回值不能解析为 JSON，可能您的 SCKEY 配置有误'
                          f'API 的返回是：\n{sc_res_raw}\n您输入的 SCKEY 为\n{sckey}')
    return_json = sc_res.get('errno')
        if errno != 0:
            raise RuntimeError(
                f'Server 酱调用失败，可能您的 SCKEY 配置有误。API 的返回是：\n{sc_res_raw}\n'
                f'您输入的 SCKEY 为\n{sckey}')

    